# **Applied Machine Learning**
# **Programming Assignment 4: Linear classifiers**
**Group 34**

**Dipti Aswal(gusaswdi@student.gu.se)**

**Atefeh Aminmoghaddam(gusamiat@student.gu.se)**

## **Exercise question:**

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

X1 = [{'city':'Gothenburg', 'month':'July'},
      {'city':'Gothenburg', 'month':'December'},
      {'city':'Paris', 'month':'July'},
      {'city':'Paris', 'month':'December'}]
Y1 = ['rain', 'rain', 'sun', 'rain']

X2 = [{'city':'Sydney', 'month':'July'},
      {'city':'Sydney', 'month':'December'},
      {'city':'Paris', 'month':'July'},
      {'city':'Paris', 'month':'December'}]
Y2 = ['rain', 'sun', 'sun', 'rain']

classifier1 = make_pipeline(DictVectorizer(), Perceptron(max_iter=10))
classifier1.fit(X1, Y1)
guesses1 = classifier1.predict(X1)
print('accuracy of the training data set 1:',accuracy_score(Y1, guesses1))

classifier2 = make_pipeline(DictVectorizer(), Perceptron(max_iter=10))
#classifier2 = make_pipeline(DictVectorizer(), LinearSVC())
classifier2.fit(X2, Y2)
guesses2 = classifier2.predict(X2)
print('accuracy of the training data set 2:',accuracy_score(Y2, guesses2))

accuracy of the training data set 1: 1.0
accuracy of the training data set 2: 0.5


**Explanation:**
The result of both training data sets is observed above. It seems that the difference between two datasets which affect the lower accuracy is related to the labels corresponding to each row which in the second dataset the labels for both two same months are different. 

The Perceptron algorithm is a two-class (binary) classification machine learning algorithm. It learns a decision boundary that separates two classes using a line in the feature space. So, it is appropriate for those problems where the classes can be linearly separable. In the second training dataset, with the labels considered for the rows, the data can not be linearly separable. 

Also the perceptron remembers patterns in sequential data, so it requires a large number in parameters to process the data.

Another point that can affect the low accuracy in machine learning models which is not involved in the case of the exercise is imbalanced classes. In this situation, training a classification model can result in the model predicting the most represented class for all inputs.





In [ ]:
from google.colab import files
uploaded = files.upload()

Saving all_sentiment_shuffled.txt to all_sentiment_shuffled.txt


## **Observations after using below approaches in implementation:**


### **Original time taken by Support Vector Classifier and Logistic Regression implementation:**
    
    Training time for SVC algorithm : 2.60 sec.
    Accuracy of SVC algorithm : 0.8326.

    Training time for LR algorithm: 4.95 sec.
    Accuracy of LR algorithm: 0.8053.

## **Bonus task 1:**
### **Using blas library for improving linear algebra performance:**
After replacing numpy operations by blas functions the time has improved considerably.

    Training time for SVC algorithm : 1.84 sec.
    Accuracy of SVC algorithm : 0.8380.

    Training time for LR algorithm: 2.10 sec.
    Accuracy of LR algorithm: 0.8452.
        
### **Remove SelectKBest:**
Using all features increased the time manifolds with slight improvement in accuracy in case of Logistic Regression.

    Training time for SVC algorithm : 9.31 sec.
    Accuracy of SVC algorithm : 0.8103.

    Training time for LR algorithm: 4.54 sec.
    Accuracy of LR algorithm: 0.8305.

### **Remove SelectKBest and add ngram_range=(1,2):**
Upon using all features and restricting only unigrams and bigrams we see that the time shoots up to almost a minute.

    Training time for SVC algorithm : 68.24 sec.
    Accuracy of SVC algorithm : 0.8401.

    Training time for LR algorithm: 26.52 sec.
    Accuracy of LR algorithm: 0.8431.

### **Using ALL features (after removing selectKbest) and implementing sparse vector operations:**
Using sparse implementation on ALL features the time reduced sligthly.
    
    Training time for SVC algorithm : 9.31 sec.
    Accuracy of SVC algorithm : 0.8103.

    Training time for LR algorithm: 4.54 sec.
    Accuracy of LR algorithm: 0.8305.
    
### **Using scaling factor on top of sparse vector implementation:**
We can see minor improvement in timings.

    Training time for SVC algorithm : 8.60 sec.
    Accuracy of SVC algorithm : 0.8103.

    Training time for LR algorithm: 4.63 sec.
    Accuracy of LR algorithm: 0.8305.



# **Bonus Task 2**
**Running the OneVsRestClassifier for both SVC and Logistic Regression**



Training time for OneVsRestClassifier strategy by using SVC algorithm : 6.14 sec.

Accuracy of OneVsRestClassifier strategy by using SVC algorithm : 0.9299.


Training time for OneVsRestClassifier strategy by using LR algorithm: 15.99 sec.

Accuracy of OneVsRestClassifier strategy by using LR algorithm: 0.8775.

**Running the OneVsOneClassifier for both SVC and Logistic Regression**

Training time for OneVsOneClassifier strategy by using SVC algorithm : 5.08 sec.

Accuracy of OneVsOneClassifier strategy by using SVC algorithm : 0.9207.


Training time for OneVsOneClassifier strategy by using LR algorithm: 16.03 sec.

Accuracy of OneVsOneClassifier strategy by using LR algorithm: 0.8775.



# **Task 1: Implementing the SVC and Logistic Regression using hinge loss and log loss respectively**

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator

class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])


class Perceptron_pegasos_SVC(LinearClassifier):

    def __init__(self, n_iter, lam):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.lam=lam

       
    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm for SVC.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        t=0

        # Implementing SVC with hinge loss:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):
                t=t+1
                eta=1/(self.lam*t)

                # Compute the output score for this instance.
                score = x.dot(self.w)
                
                # Update weight using hinge loss function
                if y*score <1:
                    self.w = (1-eta*self.lam)*self.w+(eta*y)*x
                else:
                    self.w=(1-eta*self.lam)*self.w

                    
class Perceptron_pegasos_LR(LinearClassifier):

    def __init__(self, n_iter, lam):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.lam = lam

       
    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm for Logistic Regression.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        t=0

        # Implementing Logistic Regression with Log loss:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):
                t=t+1
                eta=1/(self.lam*t)
                sigmoid = 1 + np.exp(y*self.w*x)
                
                # Update weight using log loss function
                self.w = (1 - eta*self.lam)*self.w + eta*y*x/sigmoid

In [ ]:
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#from aml_perceptron import Perceptron, SparsePerceptron

# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y


if __name__ == '__main__':
    
    # Read all the documents.
    X, Y = read_data('all_sentiment_shuffled.txt')
    
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)
    # Setting number of iterations to 10
    n_iter = 10
    
    # Setting lamda as 1/number of instances in training set
    lamda = 1/len(Xtrain)
    
    # Set up the preprocessing steps and SVC classifier.
    pipeline = make_pipeline(
        TfidfVectorizer(),
        SelectKBest(k=1000),
        Normalizer(),

        # NB that this is our Perceptron, not sklearn.linear_model.Perceptron
        Perceptron_pegasos_SVC(n_iter, lamda)  
    )

    # Train the SVC classifier.
    t0 = time.time()
    pipeline.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('Training time for SVC algorithm : {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline.predict(Xtest)
    print('Accuracy of SVC algorithm : {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

    # Set up the preprocessing steps and the LR classifier.
    pipeline1 = make_pipeline(
        TfidfVectorizer(),
        SelectKBest(k=1000),
        Normalizer(),

        # NB that this is our Perceptron, not sklearn.linear_model.Perceptron
        Perceptron_pegasos_LR(n_iter, lamda)  
    )
    # Train the LR classifier.
    t0 = time.time()
    pipeline1.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('\n\nTraining time for LR algorithm: {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline1.predict(Xtest)
    print('Accuracy of LR algorithm: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time for SVC algorithm : 2.60 sec.
Accuracy of SVC algorithm : 0.8326.


Training time for LR algorithm: 4.95 sec.
Accuracy of LR algorithm: 0.8053.
